In [1]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [13]:
from camel.agents import ChatAgent
from camel.configs import DeepSeekConfig, ZhipuAIConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.toolkits import (
    SearchToolkit
)
from camel.messages import BaseMessage

In [3]:
def add(a: int, b: int) -> int:
    r"""Adds two numbers.

    Args:
        a (int): The first number to be added.
        b (int): The second number to be added.

    Returns:
        integer: The sum of the two numbers.
    """
    return a + b

def sub(a: int, b: int) -> int:
    r"""Do subtraction between two numbers.

    Args:
        a (int): The minuend in subtraction.
        b (int): The subtrahend in subtraction.

    Returns:
        integer: The result of subtracting :obj:`b` from :obj:`a`.
    """
    return a - b

Add these 2 customized functions as CAMEL’s FunctionTool list:

In [4]:
from camel.toolkits import FunctionTool

MATH_FUNCS: list[FunctionTool] = [
    FunctionTool(func) for func in [add,sub]
]

Then you can add the tool from CAMEL and the one defined by yourself to the tool list:

In [7]:
tool_lists = [
    FunctionTool(SearchToolkit().search_duckduckgo),
    *MATH_FUNCS
]

Next let’s set the parameters to the agent and initianize ChatAgent to call the tool:

In [31]:
model = ModelFactory.create(
    model_platform=ModelPlatformType.ZHIPU,
    model_type=ModelType.GLM_4_FLASH,
    model_config_dict=ZhipuAIConfig().as_dict(),
    api_key="6e943baaede04a749bbe3744c890a338.LAwE3d88i1vmcrdh"
)

# Set message for the assistant
assistant_sys_msg =  """You are a helpful assistant to do search task."""

# set the agent
agent = ChatAgent(
    assistant_sys_msg,
    model=model,
    tools=tool_lists
)

Here we define two test prompts for the agent, asking about the facts about University of Oxford. Here the agent needs to take advantage of the searching capability to know when University of Oxford is founded and the calculating skills to obtain the estimated age of the Uni.

In [26]:
# Set prompt for the search task
prompt_search = ("""When was University of Oxford set up""")
# Set prompt for the calculation task
prompt_calculate = ("""Assume now is 2024 in the Gregorian calendar, University of Oxford was set up in 1096, estimate the current age of University of Oxford""")

# Convert the two prompt as message that can be accepted by the Agent
user_msg_search = BaseMessage.make_user_message(role_name="User", content=prompt_search)
user_msg_calculate = BaseMessage.make_user_message(role_name="User", content=prompt_calculate)

In [27]:
# Get response
assistant_response_calculate = agent.step(user_msg_calculate)

In [28]:
assistant_response_calculate

ChatAgentResponse(msgs=[BaseMessage(role_name='Assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='To estimate the current age of the University of Oxford, we can simply subtract the year it was founded from the current year.\n\n2024 (current year) - 1096 (year founded) = 928 years\n\nSo, as of 2024, the University of Oxford would be approximately 928 years old.', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': '20250309114127ad58e82c51eb4c3c', 'usage': {'completion_tokens': 68, 'prompt_tokens': 49, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 55, 'tool_calls': [], 'external_tool_call_request': None})

In [29]:
assistant_response_search = agent.step(user_msg_search)

In [30]:
assistant_response_search

ChatAgentResponse(msgs=[BaseMessage(role_name='Assistant', role_type=<RoleType.ASSISTANT: 'assistant'>, meta_dict={}, content='The University of Oxford was set up in the year 1096. It is one of the oldest universities in the world and is often considered the oldest in the English-speaking world.', video_bytes=None, image_list=None, image_detail='auto', video_detail='low', parsed=None)], terminated=False, info={'id': '202503091141417e1f4a58428b4e12', 'usage': {'completion_tokens': 38, 'prompt_tokens': 126, 'total_tokens': 164, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'termination_reasons': ['stop'], 'num_tokens': 140, 'tool_calls': [], 'external_tool_call_request': None})